# Extract Information

In [1]:
import pandas as pd
import json
import os
from sqlalchemy import create_engine, text

In [2]:
db_string = ""
db = create_engine(db_string)
connection = db.connect()
print("Connected!")
query = "SELECT * FROM outcome"
df = pd.read_sql_query(query, db)
connection.close()
db.dispose()

Connected!


In [3]:
df.to_csv("raw.csv", index= False)

In [2]:
raw_data = pd.read_csv("raw.csv")

### Personal Information

In [3]:
columns_to_extract = ['id', 'age', 'gender', 'levelofeducation', 'occupation', 'topicnumber', 'conditionnumber']
PATH = 'Data/personal_data.csv'
df_personal = raw_data[columns_to_extract]

if os.path.exists(PATH):
    old = pd.read_csv(PATH)
    new = pd.concat([df_personal, old])
    new.to_csv(PATH, index=False)
else:
    df_personal.to_csv(PATH, index=False)

### General

In [4]:
result_general = pd.DataFrame()

for index, row in raw_data.iterrows():
    id = row['id']
    general = row['general']

    dict = pd.read_json(general, typ='series')
    first = dict['first']
    second = dict['second']
    third = dict['third']
    fourth = dict['fourth']

    df = pd.DataFrame({'id':[id],first['question']: first['answer'], second['question']: second['answer'], third['question']: third['answer'], fourth['question']: fourth['answer']})

    result_general = pd.concat([result_general, df], ignore_index=True)

result_general.to_csv('Data/general.csv', index=False)


### General END

In [5]:
result_general_end = pd.DataFrame()

for index, row in raw_data.iterrows():
    id = row['id']
    general = row['general_end']

    dict = pd.read_json(general, typ='series')
    first = dict['first']

    df = pd.DataFrame({'id':[id],first['question']: first['answer']})

    result_general_end = pd.concat([result_general_end, df], ignore_index=True)

result_general_end.to_csv('Data/general_end.csv', index=False)

### Timestamps


In [6]:
result_topic_timeStamps = pd.DataFrame()

level = {
  "1":["englisch_leicht","deutsch_leicht","englisch_schwer","deutsch_schwer"],
  "2":["deutsch_leicht","englisch_schwer","deutsch_schwer","englisch_leicht"],
  "3":["englisch_schwer","deutsch_schwer","englisch_leicht","deutsch_leicht"],
  "4":["deutsch_schwer","englisch_leicht","deutsch_leicht","englisch_schwer"]
}

topic = {
  "1":["A","B","C","D"],
  "2":["B","C","D","A"],
  "3":["C","D","A","B"],
  "4":["D","A","B","C"]
}

def get_topic(topic_now, order_list):
    result_list = {}
    for index, topic in enumerate(topic_now):
        result_list[topic] = order_list[index]
    return result_list

def get_condition(level_now, order_list):
    result_list = {}
    for index, condition in enumerate(level_now):
        result_list[condition] = order_list[index]
    return result_list

for index, row in raw_data.iterrows():
    id = row['id']
    timestamp = row['time_stamp']
    topic_number = row['topicnumber']
    condition = row['conditionnumber']

    level_now = level[str(condition)]
    topic_now = topic[str(topic_number)]


    timestamp_dict = pd.read_json(timestamp, typ='series')

    one = timestamp_dict['one']
    two = timestamp_dict['two']
    three = timestamp_dict['three']
    four = timestamp_dict['four']
    order_list = [one,two,three,four]
    topic_list = get_topic(topic_now, order_list)
    condition_list = get_condition(level_now,order_list)



    df = pd.DataFrame({'participant_id': [id], 'text_one':[one],'text_two':[two], 'text_three':[three], 'text_four':[four], "A":[topic_list["A"]],"B":[topic_list["B"]],"C":[topic_list["C"]],"D":[topic_list["D"]],"englisch_leicht":[condition_list["englisch_leicht"]],"deutsch_leicht":[condition_list["deutsch_leicht"]],"englisch_schwer":[condition_list["englisch_schwer"]],"deutsch_schwer":[condition_list["deutsch_schwer"]]})

    result_topic_timeStamps = pd.concat([result_topic_timeStamps, df], ignore_index=True)

result_topic_timeStamps.to_csv('Data/topic_timestamps.csv', index=False)

### Pre_Topic

In [7]:
def get_key(topic, list):
    index_of_b = list.index(topic)
    if index_of_b == 0:
        return "pre_one"
    elif index_of_b == 1:
        return "pre_two"
    elif index_of_b == 2:
        return "pre_three"
    elif index_of_b == 3:
        return "pre_four"


def get_Topic_Pre(topic):
    df_result = pd.DataFrame()
    for index, row in raw_data.iterrows():
        data = row['condition']
        id = row['id']
        df = pd.read_json(data, typ='series')
        topic_list = df['topic']
        key = get_key(topic, topic_list)
        data = row[key]
        df_json = pd.read_json(data, typ='series')
        extracted_info = {}
        extracted_info['id'] = [id]
        for key, value in df_json.items():
            extracted_info[value['question']] = value['answer']
        dataframe = pd.DataFrame(extracted_info)
        df_result = pd.concat([df_result, dataframe], ignore_index=True)

    df_result.to_csv(f"Data/pre_{topic}.csv",index = False)

ls = ['A','B','C','D']

for i in ls:
    get_Topic_Pre(i)

### POST - Topic

In [8]:
def get_condition(topic, list, condition_list):
    index_of_b = list.index(topic)
    return condition_list[index_of_b]
    
def get_key_post(topic, list):
    index_of_b = list.index(topic)
    if index_of_b == 0:
        return "post_one"
    elif index_of_b == 1:
        return "post_two"
    elif index_of_b == 2:
        return "post_three"
    elif index_of_b == 3:
        return "post_four"
    
level = {
  "1":["englisch_leicht","deutsch_leicht","englisch_schwer","deutsch_schwer"],
  "2":["deutsch_leicht","englisch_schwer","deutsch_schwer","englisch_leicht"],
  "3":["englisch_schwer","deutsch_schwer","englisch_leicht","deutsch_leicht"],
  "4":["deutsch_schwer","englisch_leicht","deutsch_leicht","englisch_schwer"]
}



def get_Topic_Post(topic):
    df_result = pd.DataFrame()
    for index, row in raw_data.iterrows():
        data = row['condition']
        id = row['id']
        df = pd.read_json(data, typ='series')
        topic_list = df['topic']
        condition_list = df['condition']
        key = get_key_post(topic, topic_list)
        condition = get_condition(topic, topic_list, condition_list)
        data = row[key]
        df_json = pd.read_json(data, typ='series')
        extracted_info = {}
        extracted_info['id'] = [id]
        for key, value in df_json.items():
            extracted_info[value['question']] = value['answer']
        extracted_info["soloCondition"] = condition
        dataframe = pd.DataFrame(extracted_info)
        df_result = pd.concat([df_result, dataframe], ignore_index=True)

    df_result.to_csv(f"Data/post_{topic}.csv",index = False)

ls = ['A','B','C','D']

for i in ls:
    get_Topic_Post(i)
